# Tecnologico de Costa Rica - Fundatec
# Programa Ciencia de Datos
## Mineria de Datos e Inteligencia de Negocios
### Prof. Lorena Zuñiga
### Estudiante Sergio Castillo Segura

### Tarea 4 - Web Scraping

In [59]:
install.packages("FactoMineR")
install.packages("dendextend")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [102]:
install.packages("purrr")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [124]:
#Importacion de modulos
library(tibble)
library(stringr)
library(rvest)
library(FactoMineR)
library(ggplot2)
library(dendextend)
library(plyr)
library(purrr)
library(ggplot2)

## Entendimiento de Datos
### Descripcion
La pagina web seleccionada incluye las estadisticas de contagios del virus COVID-19 en todo el mundo.
Contiene una tabla que, entre otros datos, incluye la cantidad de infectados por pais, la cantidad de muertes y la cantidad de personas recuperadas de la enfermedad.

### Objetivo de negocio
Identificar los principales paises donde se deben enfocar los recursos medicos con el fin de tomar mejores medidas preventorias de contagio del virus COVID-19

### Objetivo de mineria de datos
Identificar un metodo de agrupamiento de los datos que permita reconocer los paises que presentan un mayor indice de crecimiento de contagios.

In [5]:
#Sitio web que se leerá
pageUrl = 'https://www.worldometers.info/coronavirus/'
pageHtml = read_html(pageUrl, encoding = 'UTF-8')  
# se extraen específicamente las tablas que contenga la página
pageTbls = html_table(pageHtml, fill = TRUE)
length(pageTbls)

[1] 1

### Exploracion de datos

In [6]:
#Revisamos la unica tabla encontrada
pageTbls

"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>
China,"80,860",+16,"3,213",14,"67,752","9,895","3,226",56.2
Italy,"24,747","+3,590","1,809",368,"2,335","20,603","1,672",409.3
Iran,"13,938","+1,209",724,113,"4,590","8,624",,165.9
S. Korea,"8,236",+150,75,3,"1,137","7,024",59,160.6
Spain,"7,845","+1,454",292,96,517,"7,036",272,167.8
Germany,"5,813","+1,214",11,2,46,"5,756",2,69.4
France,"5,423",+924,127,36,12,"5,284",400,83.1
USA,"3,737",+794,68,11,73,"3,596",10,11.3
Switzerland,"2,217",+842,14,1,4,"2,199",,256.2


### Calidad de datos
La tabla esta completa, para los valores NA (not available) se asumen como 0 pues al temas estadistico es lo mismo.
Solo se incluyen los paises que han presentado estadisticas hasta el momento y son datos oficiales por lo que se pueden tomar como ciertos.

### Limpieza de Datos

In [20]:
#Renombramos las columnas a nombres mas sencillos y eliminamos las que no nos interesan
data = pageTbls[[1]]
colnames(data)
names(data)[names(data) == "Country,Other"] = "Country"
names(data)[names(data) == "Serious,Critical"] = "Serious"
data  = data[-9]  #Eliminamos la ultima columna
head(data)

[1] "Country,Other"    "TotalCases"       "NewCases"         "TotalDeaths"     
[5] "NewDeaths"        "TotalRecovered"   "ActiveCases"      "Serious,Critical"
[9] "Tot Cases/1M pop"

Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Serious
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
China,"80,860",+16,"3,213",14,"67,752","9,895","3,226"
Italy,"24,747","+3,590","1,809",368,"2,335","20,603","1,672"
Iran,"13,938","+1,209",724,113,"4,590","8,624",
S. Korea,"8,236",+150,75,3,"1,137","7,024",59
Spain,"7,845","+1,454",292,96,517,"7,036",272
Germany,"5,813","+1,214",11,2,46,"5,756",2


In [25]:
#Ahora hay que convertir las columnas que no fueron detectadas como numericas a numericas, implica eliminar las comas de miles
data$TotalCases = str_remove_all(data$TotalCases, ",")
data$NewCases = str_remove_all(data$NewCases, "[\\+,]")
data$TotalDeaths = str_remove_all(data$TotalDeaths, ",")
data$TotalRecovered = str_remove_all(data$TotalRecovered, ",")
data$ActiveCases = str_remove_all(data$ActiveCases, ",")
data$Serious = str_remove_all(data$Serious, ",")
head(data)

Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Serious
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
China,80860,16,3213,14,67752,9895,3226
Italy,24747,3590,1809,368,2335,20603,1672
Iran,13938,1209,724,113,4590,8624,
S. Korea,8236,150,75,3,1137,7024,59
Spain,7845,1454,292,96,517,7036,272
Germany,5813,1214,11,2,46,5756,2


#### No es necesario crear nuevos datos/atributos, pero si será necesario transformar las columnas textuales a numericas para su posterior procesamiento

In [38]:
#Tranformamos el dataset a numerico
data = transform(data, 
      TotalCases = as.numeric(TotalCases),
      NewCases = as.numeric(NewCases),
      TotalDeaths = as.numeric(TotalDeaths),
      TotalRecovered = as.numeric(TotalRecovered),
      ActiveCases = as.numeric(ActiveCases),
      Serious = as.numeric(Serious)
)

In [39]:
#Removemos la ultima fila con los totales
data = data[1:nrow(data)-1,]
tail(data)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Serious
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
152,Mongolia,1,NA,NA,0,NA,1,NA
153,St. Barth,1,NA,NA,0,NA,1,NA
154,St. Vincent Grenadines,1,NA,NA,0,NA,1,NA
155,Suriname,1,NA,NA,0,NA,1,NA
156,Eswatini,1,NA,NA,0,NA,1,NA
157,Togo,1,NA,NA,0,NA,1,NA


In [40]:
#Reemplazamos todos los NA por 0 ya que no afectan en nada a la estadistica
data[is.na(data)] = 0
data

Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Serious
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
China,80860,16,3213,14,67752,9895,3226
Italy,24747,3590,1809,368,2335,20603,1672
Iran,13938,1209,724,113,4590,8624,0
S. Korea,8236,150,75,3,1137,7024,59
Spain,7845,1454,292,96,517,7036,272
Germany,5813,1214,11,2,46,5756,2
France,5423,924,127,36,12,5284,400
USA,3737,794,68,11,73,3596,10
Switzerland,2217,842,14,1,4,2199,0


## Modelado de datos

In [53]:
#Primero escalamos los datos para asegurarnos que estamos usando las mismas escalas, esto es muy importante para el clustering
scaled = as.data.frame(scale(data[2:7]))
data_scaled = cbind(data[,1], scaled)
names(data_scaled)[1] = "Country"
data_scaled

Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
China,11.595899920,-0.186950209,10.60026662,0.30499218,12.412440710,4.30028369
Italy,3.439991239,9.947445263,5.90761034,11.48629518,0.339532968,9.22407671
Iran,1.868925409,3.195907597,2.28116300,3.43196675,0.755700125,3.71584767
S. Korea,1.040151484,0.193018731,0.11197929,-0.04244944,0.118438398,2.98012962
Spain,0.983320441,3.890626928,0.83726876,2.89501152,0.004015498,2.98564751
Germany,0.687973437,3.210085542,-0.10193096,-0.07403504,-0.082908994,2.39707307
France,0.631287742,2.387764701,0.28578138,0.99987542,-0.089183798,2.18003625
USA,0.386231124,2.019138117,0.08858286,0.21023537,-0.077926062,1.40385371
Switzerland,0.165302262,2.155246394,-0.09190392,-0.10562064,-0.090660223,0.76147989


### Seleccion de tecnicas
Las tecnicas a probar seran clustering jerarquico y kmeans.

### Clustering jerarquico - Modelo 1
En este modelo primero se creara una matriz de distancias, luego se generara un dendograma a partir de los clusters obtenidos

In [55]:
#Matrix de distancias
dist_countries <- dist(data_scaled, method = "euclidean")

Warning message in dist(data_scaled, method = "euclidean"):
“NAs introduced by coercion”

In [56]:
#Extraccion de clusters
hc_countries <- hclust(dist_countries, method = "average")

In [57]:
#Generacion de dendograma
dend_countries <- as.dendrogram(hc_countries)
plot(dend_countries, main = "Clustering entre paises", ann = FALSE)

In [93]:
#El resultado no está mal, se ve una agrupacion importante
#Utilizaremos colores para identificar la cantidad de clusters con altura de 3
dend_countries_colored <- color_branches(dend_countries, h = 3)
plot(dend_countries_colored, label = dend_countries_colored$Country)

In [130]:
#Agruparemos de igual forma con altura de 3 para obtener la cantidad de grupos importantes
clusters_h2 <- cutree(hc_countries, h = 3)

In [138]:
#Revisamos la cantidad de paises que quedaron en cada grupo
x = count(clusters_h2)
xx = barplot(x$freq, names.arg = x$x, ylim = c(0, 200))
text(x = xx, y = x$freq, label = x$freq, pos = 3, cex = 0.8, col = "blue")

#### Esto nos dice que hay 6 grupos importantes, ahora adjuntamos los grupos a la data de paises para ver la relacion

In [95]:
cnt_clusters = cbind(data, clusters_h2)
cnt_clusters

Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,Serious,clusters_h2
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
China,80860,16,3213,14,67752,9895,3226,1
Italy,24747,3590,1809,368,2335,20603,1672,2
Iran,13938,1209,724,113,4590,8624,0,3
S. Korea,8236,150,75,3,1137,7024,59,4
Spain,7845,1454,292,96,517,7036,272,3
Germany,5813,1214,11,2,46,5756,2,5
France,5423,924,127,36,12,5284,400,5
USA,3737,794,68,11,73,3596,10,5
Switzerland,2217,842,14,1,4,2199,0,5


In [98]:
#Y mostramos los paises que pertenecen a cada grupo
aggregate(Country ~ clusters_h2, data = cnt_clusters, toString)

clusters_h2,Country
<int>,<chr>
1,China
2,Italy
3,"Iran, Spain"
4,S. Korea
5,"Germany, France, USA, Switzerland"
6,"UK, Norway, Netherlands, Sweden, Belgium, Denmark, Austria, Japan, Diamond Princess, Malaysia, Qatar, Canada, Greece, Australia, Czechia, Portugal, Finland, Singapore, Slovenia, Bahrain, Israel, Brazil, Iceland, Estonia, Ireland, Hong Kong, Philippines, Romania, Egypt, Poland, Iraq, Saudi Arabia, Indonesia, Thailand, India, Kuwait, San Marino, Lebanon, UAE, Luxembourg, Chile, Peru, Russia, Slovakia, South Africa, Taiwan, Vietnam, Pakistan, Bulgaria, Brunei, Croatia, Algeria, Serbia, Argentina, Panama, Mexico, Albania, Palestine, Ecuador, Costa Rica, Colombia, Georgia, Cyprus, Hungary, Latvia, Morocco, Belarus, Armenia, Senegal, Bosnia and Herzegovina, Azerbaijan, Moldova, Oman, Malta, Tunisia, North Macedonia, Sri Lanka, Turkey, Venezuela, Afghanistan, Lithuania, Maldives, Cambodia, Macao, Dominican Republic, Faeroe Islands, Jordan, Bolivia, Jamaica, Martinique, Kazakhstan, New Zealand, French Guiana, Liechtenstein, Paraguay, Réunion, Ghana, Uruguay, Andorra, Bangladesh, Puerto Rico, Rwanda, Guyana, Cameroon, Ivory Coast, Cuba, Ethiopia, Uzbekistan, Ukraine, Burkina Faso, Channel Islands, French Polynesia, Guadeloupe, Guam, Honduras, Kenya, Seychelles, Monaco, Nigeria, Aruba, Curaçao, DRC, Namibia, Saint Lucia, Saint Martin, Trinidad and Tobago, Guatemala, Sudan, Nepal, Antigua and Barbuda, Bahamas, Bhutan, Cayman Islands, CAR, Congo, Equatorial Guinea, Gabon, Gibraltar, Guinea, Vatican City, Mauritania, Mayotte, Mongolia, St. Barth, St. Vincent Grenadines, Suriname, Eswatini, Togo"


##### Como se puede observar, los grupos 1, 2 y 4 nos muestran paises aislados que llevan su propia batalla y ya sea que estan controlando la infeccion muy bien como China y Sur Korea, o que siguen descontrolados desproporcionalmente como Italia (de acuerdo a la cantidad de casos nuevos). Por otro lado el grupo 3 muestra paises cuya cantidad de contagios sigue creciendo a gran medida pero que quiza se puedan contener con metodos mas eficientes, mientras que el grupo 5 muestra otros posibles candidatos a intervencion, los paises del grupo 6 aun tienen muestras muy bajas de contagios que podrian tratarse por otros metodos.
#### En realidad como resultados medicos no tengo mucha experiencia por lo que habria que ver si a la organizacion medica esta agrupacion le sirve de algo.
#### Ahora realizaremos el estudio usando kmeans

### K-Means - modelo 2
Se estimara el mejor numero de clusters usando el metodo elbow.

In [106]:
tot_withinss <- map_dbl (1:10, 
    function(k) { 
        model <- kmeans(x = data_scaled[-1], centers = k)
        model$tot.withinss
    }
)

elbow_df <- data.frame(k = 1:10, tot_withinss = tot_withinss)

elbow_df %>%
  ggplot(aes(x = k, y = tot_withinss)) +
  geom_line() +
  scale_x_continuous(breaks = 1:10)

#### k=2 tiene una curva importante que reduce la cantidad de errores
#### Pero k=4 tiene una curva aun mas pronunciada, asi que lo haremos con 4.

In [122]:
countries_k3 = kmeans(data_scaled[-1], centers = 4)

In [140]:
#Verificamos la cantidad de paises que quedaron en cada grupo
x = count(countries_k3$cluster)
xx = barplot(x$freq, names.arg = x$x, ylim = c(0, 200))
text(x = xx, y = x$freq, label = x$freq, pos = 3, cex = 0.8, col = "blue")

In [119]:
countries_k3_final  = cbind(data_scaled, countries_k3$cluster)
countries_k3_final

Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,countries_k3$cluster
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
China,11.595899920,-0.186950209,10.60026662,0.30499218,12.412440710,4.30028369,1
Italy,3.439991239,9.947445263,5.90761034,11.48629518,0.339532968,9.22407671,2
Iran,1.868925409,3.195907597,2.28116300,3.43196675,0.755700125,3.71584767,4
S. Korea,1.040151484,0.193018731,0.11197929,-0.04244944,0.118438398,2.98012962,4
Spain,0.983320441,3.890626928,0.83726876,2.89501152,0.004015498,2.98564751,4
Germany,0.687973437,3.210085542,-0.10193096,-0.07403504,-0.082908994,2.39707307,4
France,0.631287742,2.387764701,0.28578138,0.99987542,-0.089183798,2.18003625,4
USA,0.386231124,2.019138117,0.08858286,0.21023537,-0.077926062,1.40385371,4
Switzerland,0.165302262,2.155246394,-0.09190392,-0.10562064,-0.090660223,0.76147989,4


In [120]:
#Y mostramos los paises que pertenecen a cada grupo
aggregate(Country ~ countries_k3$cluster, data = countries_k3_final, toString)

countries_k3$cluster,Country
<int>,<chr>
1,China
2,Italy
3,"UK, Norway, Netherlands, Sweden, Belgium, Denmark, Austria, Japan, Diamond Princess, Malaysia, Qatar, Canada, Greece, Australia, Czechia, Portugal, Finland, Singapore, Slovenia, Bahrain, Israel, Brazil, Iceland, Estonia, Ireland, Hong Kong, Philippines, Romania, Egypt, Poland, Iraq, Saudi Arabia, Indonesia, Thailand, India, Kuwait, San Marino, Lebanon, UAE, Luxembourg, Chile, Peru, Russia, Slovakia, South Africa, Taiwan, Vietnam, Pakistan, Bulgaria, Brunei, Croatia, Algeria, Serbia, Argentina, Panama, Mexico, Albania, Palestine, Ecuador, Costa Rica, Colombia, Georgia, Cyprus, Hungary, Latvia, Morocco, Belarus, Armenia, Senegal, Bosnia and Herzegovina, Azerbaijan, Moldova, Oman, Malta, Tunisia, North Macedonia, Sri Lanka, Turkey, Venezuela, Afghanistan, Lithuania, Maldives, Cambodia, Macao, Dominican Republic, Faeroe Islands, Jordan, Bolivia, Jamaica, Martinique, Kazakhstan, New Zealand, French Guiana, Liechtenstein, Paraguay, Réunion, Ghana, Uruguay, Andorra, Bangladesh, Puerto Rico, Rwanda, Guyana, Cameroon, Ivory Coast, Cuba, Ethiopia, Uzbekistan, Ukraine, Burkina Faso, Channel Islands, French Polynesia, Guadeloupe, Guam, Honduras, Kenya, Seychelles, Monaco, Nigeria, Aruba, Curaçao, DRC, Namibia, Saint Lucia, Saint Martin, Trinidad and Tobago, Guatemala, Sudan, Nepal, Antigua and Barbuda, Bahamas, Bhutan, Cayman Islands, CAR, Congo, Equatorial Guinea, Gabon, Gibraltar, Guinea, Vatican City, Mauritania, Mayotte, Mongolia, St. Barth, St. Vincent Grenadines, Suriname, Eswatini, Togo"
4,"Iran, S. Korea, Spain, Germany, France, USA, Switzerland"


##### Como podemos ver la agrupacion de hecho es similar al metodo jerarquico, lo que yo podria entender aca es que China e Italia son casos aislados que deben seguir sus propios metodos pero que se deberia poner atencion en el grupo 4 el cual incluye los paises que van detras de esos paises pero que un posible enfoque de recursos podria tener mejores efectos. Los paises del grupo 4, al igual que el modelo jerarquico, indica que aun tiene pocas incidencias y  deberan ser abordados una vez se demuestre que las medidas tomadas en los paises del grupo 4 funcionen.
#### Por lo menos ese es mi criterio no medico, reitero, habria que ver si a alguna organizacion medica esto le hace sentido.
